## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-15-20-47-44 +0000,bbc,Adolescence star Owen Cooper becomes Emmys' yo...,https://www.bbc.com/news/articles/cjelpe2y3plo
1,2025-07-15-20-41-00 +0000,wsj,"Inflation picked up in June, a potential sign ...",https://www.wsj.com/economy/inflation-hit-2-7-...
2,2025-07-15-20-29-02 +0000,nyt,Men Who Cut Down Beloved Tree in England Get M...,https://www.nytimes.com/2025/07/15/world/europ...
3,2025-07-15-20-28-43 +0000,cbc,9 First Nations ask court to strike down feder...,https://www.cbc.ca/news/canada/sudbury/first-n...
4,2025-07-15-20-21-13 +0000,nyt,Republicans in Congress Shift to Backing Ukrai...,https://www.nytimes.com/2025/07/15/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
44,trump,20
42,ukraine,8
146,tariffs,8
18,tariff,7
175,china,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
67,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...,58
70,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o,48
53,2025-07-15-09-38-38 +0000,bbc,'Not our war' - Trump's Nato weapons deal for ...,https://www.bbc.com/news/articles/c14e2ydv4d6o,42
48,2025-07-15-11-15-55 +0000,nyt,E.U. Nations Plan Retaliatory Tariffs Against ...,https://www.nytimes.com/2025/07/15/world/europ...,41
64,2025-07-15-04-30-50 +0000,bbc,China growth beats expectations as Trump tarif...,https://www.bbc.com/news/articles/c20r461g61xo,39


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
67,58,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...
70,28,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o
28,27,2025-07-15-14-57-21 +0000,bbc,Two dead after flash flooding in New Jersey an...,https://www.bbc.com/news/articles/c0j42xy47q7o
1,20,2025-07-15-20-41-00 +0000,wsj,"Inflation picked up in June, a potential sign ...",https://www.wsj.com/economy/inflation-hit-2-7-...
12,18,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...
15,17,2025-07-15-18-27-07 +0000,nyt,Canada Wildfire Smoke Triggers Air Quality Ale...,https://www.nytimes.com/2025/07/14/world/canad...
18,15,2025-07-15-17-14-09 +0000,bbc,Thousands of Afghans were moved to UK in secre...,https://www.bbc.com/news/articles/cvg8zy78787o
3,13,2025-07-15-20-28-43 +0000,cbc,9 First Nations ask court to strike down feder...,https://www.cbc.ca/news/canada/sudbury/first-n...
61,13,2025-07-15-06-42-06 +0000,bbc,"As theories swirl about Air India crash, key d...",https://www.bbc.com/news/articles/cp3lpdqp7x3o
16,13,2025-07-15-18-11-29 +0000,bbc,French PM looks to scrap two public holidays i...,https://www.bbc.com/news/articles/c2d02dz6gedo


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
